<h2><font color='green'>This is Tensorflow(2.4) implementation of Mask RCNN which is used for Instance Segmentation</h2>

**<h3><font color='red'>If You Find It Useful, Please Don't Forget to Upvote</h3>**
    
**<h2>Inference Notebook:</h2>**
https://www.kaggle.com/susnato/mask-rcnn-inference-in-tensorflow

**<h3><font color='green'>First Install Requirements and Download the coco-weights</h3>**

In [ ]:
!pip install keras==2.4.0
!pip install tensorflow==2.4.0
!pip install -U scikit-image==0.16.2
!pip install ../input/leekunhee-mask-rcnn/Mask_RCNN-master

###coco-weights
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5

**<h3><font color='green'>Imports</h3>**

In [ ]:
import os
import keras
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from mrcnn import config, utils
import matplotlib.pyplot as plt 
from numpy import zeros, asarray
from mrcnn import model as modellib

%matplotlib inline

seed=42
np.random.seed(seed)
tf.random.set_seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

train_fol = '../input/sartorius-cell-instance-segmentation/train'
test_fol = '../input/sartorius-cell-instance-segmentation/test'
train_csv = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
train_csv = train_csv.drop(['cell_type', 'plate_time', 'sample_date', 'sample_id', 'elapsed_timedelta', 'height', 'width'], axis = 1)
sub_csv = pd.read_csv('../input/sartorius-cell-instance-segmentation/sample_submission.csv')

print(tf.__version__, keras.__version__, tf.keras.__version__)

**<h3><font color='gray'>SPLIT THE DATASET (80-20)</h3>**

In [ ]:
os.makedirs('./dataset', exist_ok=True)
os.makedirs('./dataset/train', exist_ok=True)
os.makedirs('./dataset/val', exist_ok=True)

t = len(os.listdir(train_fol))#
ids = np.array(os.listdir(train_fol))
#80-20
ix = np.arange(t)
np.random.shuffle(ix)
trix = ids[ix[:484]]#80%
vlix = ids[ix[484:]]#20%
[shutil.copyfile(os.path.join(train_fol, etrix), f'./dataset/train/{etrix}') for etrix in trix]
[shutil.copyfile(os.path.join(train_fol, evlix), f'./dataset/val/{evlix}') for evlix in vlix]

print(f"no of files in train: {len(os.listdir('./dataset/train/'))}, val: {len(os.listdir('./dataset/val/'))}")

**<h3><font color='purple'>Change the config as required</h3>**

In [ ]:
class KaggleSartoriusConfig(config.Config):
    NAME = "kaggle_Sartorius_cfg"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 2
    STEPS_PER_EPOCH = 484
    VALIDATION_STEPS = 122
    USE_MINI_MASK = False
#Display The Config
kaggle_sartorius_model_config = KaggleSartoriusConfig()
kaggle_sartorius_model_config.display()

**<h3><font color='orange'>Define the Custom Dataset for with load_mask</h3>**

In [ ]:
class KaggleSartoriusDataset(utils.Dataset):
    def load_dataset(self, dataset_dir, is_train=True):
        # Adds information (image ID, image path, and annotation file path) about each image in a dictionary.
        self.add_class("dataset", 1, "cell")
        if is_train:
            images_dir = dataset_dir+'/train/' 
        else:
            images_dir = dataset_dir+'/val/'

        for filename in os.listdir(images_dir):
            image_id = filename[:-4]
            img_path = images_dir + filename
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=None)

    # Loads the binary masks for an image.
    def load_mask(self, image_id):
        info = self.image_info[image_id]
        
        train_annots = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
        annot = train_annots[train_annots.id==info['id']].annotation.values

        class_ids = list()
        masks = np.zeros([520*704, len(annot)], dtype=np.uint8)
        for i, rle in enumerate(annot):
            rle = np.array(rle.split(' ')).reshape(-1, 2)
            for r in rle:
                masks[int(r[0]):int(r[0])+int(r[1]), i] = 1
            class_ids.append(self.class_names.index('cell'))
        masks = masks.reshape(520, 704, len(annot))
        return masks, asarray(class_ids, dtype='int32')

#mAP Callback
class MeanAveragePrecisionCallback(tf.keras.callbacks.Callback):
    def __init__(self, train_model: modellib.MaskRCNN, inference_model: modellib.MaskRCNN, dataset: utils.Dataset,
                 dataset_limit: int = None, verbose: int = 1):
        super().__init__()
        self.train_model = train_model
        self.inference_model = inference_model
        self.dataset = dataset
        #self.calculate_at_every_X_epoch = calculate_at_every_X_epoch
        self.dataset_limit = len(self.dataset.image_ids)
        if dataset_limit is not None:
            self.dataset_limit = dataset_limit
        self.dataset_image_ids = self.dataset.image_ids.copy()

        if inference_model.config.BATCH_SIZE != 1:
            raise ValueError("This callback only works with the bacth size of 1")

        self._verbose_print = print if verbose > 0 else lambda *a, **k: None

    def on_epoch_end(self, epoch, logs=None):
        #if epoch > 0 and epoch % self.calculate_at_every_X_epoch == 0:
        self._verbose_print("Calculating mAP...")
        self._load_weights_for_model()

        mAPs = self._calculate_mean_average_precision()
        mAP = np.mean(mAPs)

        if logs is not None:
            logs["val_mean_average_precision"] = mAP

        self._verbose_print("mAP at epoch {0} is: {1}".format(epoch+1, mAP))

        super().on_epoch_end(epoch, logs)

    def _load_weights_for_model(self):
        last_weights_path = self.train_model.find_last()
        self._verbose_print("Loaded weights for the inference model (last checkpoint of the train model): {0}".format(
            last_weights_path))
        self.inference_model.load_weights(last_weights_path,
                                          by_name=True)

    def _calculate_mean_average_precision(self):
        mAPs = []

        # Use a random subset of the data when a limit is defined
        np.random.shuffle(self.dataset_image_ids)

        for image_id in self.dataset_image_ids:
            image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(self.dataset, self.inference_model.config,
                                                                             image_id)
            #molded_images = np.expand_dims(mold_image(image, self.inference_model.config), 0)
            results = self.inference_model.detect([image], verbose=0)
            r = results[0]
            AP_range = utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask, r["rois"],
                                           r["class_ids"], r["scores"], r['masks'], verbose=0)
            #range : Default is 0.5 to 0.95 with increments of 0.05 if you want to view the AP at each
            #        range then use verbose=1
            mAPs.append(AP_range)

        return np.array(mAPs)

**<h3><font color='brown'>BEGIN THE TRAINING AND SAVE THE WEIGHTS</h3>**

In [ ]:
import logging
logging.getLogger('tensorflow').disabled = True

In [ ]:
# Define Train Set 
n_epochs = 6
train_dataset = KaggleSartoriusDataset()
train_dataset.load_dataset(dataset_dir='./dataset', is_train=True)
train_dataset.prepare()

# Define Validation Set
validation_dataset = KaggleSartoriusDataset()
validation_dataset.load_dataset(dataset_dir='./dataset', is_train=False)
validation_dataset.prepare()

# Build the Mask R-CNN Model Architecture
train_model = modellib.MaskRCNN(mode='training', 
                                 model_dir='./trained_mask_rcnn/', 
                                 config=kaggle_sartorius_model_config)
infer_model = modellib.MaskRCNN(mode="inference", 
                                 model_dir='./infer_mask_rcnn/',
                                 config=kaggle_sartorius_model_config)
mean_average_precision_callback = MeanAveragePrecisionCallback(train_model, infer_model, validation_dataset)

#load coco-weights
train_model.load_weights(filepath='./mask_rcnn_coco.h5', 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
train_model.train(train_dataset=train_dataset, 
                  val_dataset=validation_dataset, 
                  learning_rate=kaggle_sartorius_model_config.LEARNING_RATE, 
                  epochs=n_epochs, 
                  custom_callbacks=[mean_average_precision_callback] ,
                  layers='heads')

model_weights_path = f'./model_{n_epochs}.h5'
train_model.keras_model.save_weights(model_weights_path)

**<h3><font color='indigo'>Inference To View How the model did</h3>**

In [ ]:
#Load Model in Inference Mode
infer_model.load_weights(train_model.find_last(), by_name=True)
print('Weights Loaded')

In [ ]:
# Test on a random image
from mrcnn import visualize
from mrcnn.model import log
from mrcnn import model as modellib

def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def visualize_model_performence(image_id :int):
    #image_id = np.random.choice(validation_dataset.image_ids)
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(validation_dataset, kaggle_sartorius_model_config, image_id,)


    print('GROUND TRUTH MASK:->', end='\n')
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                validation_dataset.class_names, figsize=(8, 8), show_bbox=False)

    results = infer_model.detect([original_image], verbose=1)
    r = results[0]
    print("MODEL OUTPUT MASK:->")
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                validation_dataset.class_names, r['scores'], ax=get_ax(),show_bbox=False)


In [ ]:
print("Id's Available:- ", validation_dataset.image_ids)

In [ ]:
visualize_model_performence(5)

In [ ]:
visualize_model_performence(20)

**<h1>Work in progress 🚧</h1>**

**<h1>TO-DO List</h1>**
<h2>1)ADD "mAP" METRIC(✓)<br>
2)A Test Notebook (✓)<br>
3)W&B Support</h2>

**<h3>references</h3>**
ref:- https://github.com/leekunhee/Mask_RCNN